In [1]:
from IPython.core.display import Markdown, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import traceback
import numpy as np
import scipy.stats as ss
import yaml
import itertools
import sys
import os
import subprocess
from os import path
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

def printmd(*args):
    display(Markdown(' '.join(map(str, args))))

In [2]:
# !ros2 run local_planning_performance_modelling compute_metrics -r "~/ds/performance_modelling/output/test_local_planning_gen_circles/*" -s

In [3]:
local_planner_color = {'dwb': 'blue', 'rpp': 'orange', 'teb': 'green'}
robot_model_color = {'hunter': 'cyan', 'turtle': 'green'}
pd.options.display.width = 500
pd.options.display.max_rows = 0
pd.options.display.float_format = '{:,.3f}'.format
plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['lines.linewidth'] = 4
# plt.rcParams['lines.markersize'] = 3
# plt.rcParams['lines.marker'] = 'o'

# fg_color = 'white'
# plt.rcParams['grid.color'] = 'gray'
# plt.rcParams['text.color'] = fg_color
# plt.rcParams['ytick.color'] = fg_color
# plt.rcParams['xtick.color'] = fg_color
# plt.rcParams['axes.labelcolor'] = fg_color
# plt.rcParams['axes.edgecolor'] = fg_color

# bg_color = "#323a48"
# plt.rcParams['figure.facecolor'] = bg_color
# plt.rcParams['axes.facecolor'] = bg_color
# plt.rcParams['legend.facecolor'] = bg_color

In [4]:
# df = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/test_local_planning_all/results.csv"))
# with open(path.expanduser("~/ds/performance_modelling/output/test_local_planning_all/results_info.yaml")) as results_info_file:
#     results_info = yaml.safe_load(results_info_file)

df_real = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results.csv"))
with open(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results_info.yaml")) as results_real_info_file:
    results_real_info = yaml.safe_load(results_real_info_file)
df_real = df_real[(df_real.amcl_alpha_factor.notna()) & (df_real.robot_model=='turtlebot3_waffle_performance_modelling')]

df_gen = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/test_local_planning_gen_circles/results.csv"))
with open(path.expanduser("~/ds/performance_modelling/output/test_local_planning_gen_circles/results_info.yaml")) as results_gen_info_file:
    results_gen_info = yaml.safe_load(results_gen_info_file)

df = df_gen.append(df_real, sort=True)

results_info = results_gen_info
results_info['run_parameter_names'] += [i for i in results_real_info['run_parameter_names'] if i not in results_info['run_parameter_names']]

# df = df[df.amcl_alpha_factor.notna()]

# turn odometry_error into alpha_1..4
results_info['run_parameter_names'] += ['alpha_1', 'alpha_2', 'alpha_3', 'alpha_4']
results_info['run_parameter_names'].remove('odometry_error')
for i in range(0, 4):
    df[f'alpha_{i+1}'] = df['odometry_error'].apply(lambda x: eval(x)[i])
del df['odometry_error']

df.loc[df.robot_model == 'turtlebot3_waffle_performance_modelling', 'robot_model'] = 'turtle'
df.loc[df.robot_model == 'hunter2', 'robot_model'] = 'hunter'

run_parameters = [c for c in list(df.columns) if c in results_info['run_parameter_names']]
metrics_versions = [c for c in list(df.columns) if '_version' in c]
everything_else = ['run_id', 'session_id', 'run_number']
metrics = [c for c in df.columns if c not in metrics_versions + run_parameters + everything_else]
metrics_and_versions = [c for c in list(df.columns) if '_version' in c or c in metrics]

cpu_time_metrics = [c for c in metrics if 'cpu_time' in c]
max_memory_metrics = [c for c in metrics if 'max_memory' in c]
odometry_metrics = [c for c in metrics if 'odometry_error_alpha_' in c]
odometry_mean_metrics = [c for c in metrics if 'odometry_error_alpha_' in c and '_mean' in c]
odometry_std_metrics = [c for c in metrics if 'odometry_error_alpha_' in c and '_std' in c]
odometry_parameters = [f'alpha_{i}' for i in range(1, 5)]
localization_update_metrics = [c for c in metrics if 'localization_update_' in c]
localization_update_mean_metrics = [c for c in metrics if 'localization_update_' in c and '_mean' in c]
localization_update_translation_mean_metrics = [c for c in metrics if 'localization_update_' in c and '_translation_' in c and '_mean' in c]
navigation_metrics = [c for c in metrics if c in ['trajectory_length', 'execution_time', 'success_rate']]

# add metrics from existing ones
df['mean_velocity'] = df['trajectory_length'] / df['execution_time']
navigation_metrics += ['mean_velocity']
metrics += ['mean_velocity']
metrics_and_versions += ['mean_velocity']

# add useful parameters
df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))
df[max_memory_metrics] = df[max_memory_metrics]/1024**2
df.max_steering_angle_deg = df.max_steering_angle_deg.fillna(90)

printmd("## Number of Runs")
printmd(f"    {len(df.run_id.unique())}")

printmd("## Run Parameters")
for name in [run_parameter_name for run_parameter_name in run_parameters if 'localization_generator_' not in run_parameter_name]:
    values = list(df[name].unique())
    printmd(f"    {name:<70}", sorted(values))

printmd("## Metrics")
for name in metrics_and_versions:
    if name in metrics_versions:
        if len(df[name].unique()) == 1:
            printmd(f"    {name:<70} {sorted(df[name].unique())}")
        else:
            printmd(f"<code><font style='background-color:yellow;font-family:monospace'>{name:<70}{sorted(df[name].unique())} </font></code> ⚠️")
    else:
        printmd(f"    {name:<70} min: {df[name].min(skipna=True):10.4g} {'avg':>15}: {df[name].mean(skipna=True):10.4g} {'max':>15}: {df[name].max(skipna=True):10.4g}]")

## Number of Runs

    2374

## Run Parameters

    amcl_alpha_factor                                                      [nan, 0.0, 0.5, 0.75, 1.0, 1.5, 2.0]

    environment_name                                                       ['7A-2', 'airlab', 'fr079', 'intel', 'mexico', 'office_b']

...

    system_cpu_time                                                        min:       3.27             avg:      155.1             max:      965.1]

    system_max_memory                                                      min:        123             avg:       1400             max:  3.126e+04]

    trajectory_length                                                      min:          0             avg:      21.15             max:      94.27]

    trajectory_length_version                                              [1]

    mean_velocity                                                          min:          0             avg:     0.2201             max:     0.4909]

In [20]:
p_gen = list(map(lambda x: path.join(path.expanduser('~/ds/performance_modelling/output/test_local_planning_gen_circles/'), x), list(df_gen[df_gen.local_planner_node == 'rpp'].run_id)))
print(p_gen)

['/home/enrico/ds/performance_modelling/output/test_local_planning_gen_circles/session_2022-01-26_14-14-27_448616_run_000000003', '/home/enrico/ds/performance_modelling/output/test_local_planning_gen_circles/session_2022-01-26_14-14-27_448616_run_000000008', '/home/enrico/ds/performance_modelling/output/test_local_planning_gen_circles/session_2022-01-26_14-14-27_448616_run_000000021', '/home/enrico/ds/performance_modelling/output/test_local_planning_gen_circles/session_2022-01-26_14-14-27_448616_run_000000041', '/home/enrico/ds/performance_modelling/output/test_local_planning_gen_circles/session_2022-01-26_14-14-27_448616_run_000000043', '/home/enrico/ds/performance_modelling/output/test_local_planning_gen_circles/session_2022-01-26_14-14-27_448616_run_000000048', '/home/enrico/ds/performance_modelling/output/test_local_planning_gen_circles/session_2022-01-26_14-14-27_448616_run_000000051', '/home/enrico/ds/performance_modelling/output/test_local_planning_gen_circles/session_2022-01-26

In [21]:
p_real = list(map(lambda x: path.join(path.expanduser('~/ds/performance_modelling/output/test_local_planning/'), x), list(df_real[df_real.local_planner_node == 'rpp'].run_id)))
print(p_real)

['/home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_10-33-14_965419_run_000000070', '/home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_13-54-20_413082_run_000000016', '/home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_16-44-01_042364_run_000000015', '/home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_16-44-01_042364_run_000000018', '/home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_16-44-01_042364_run_000000029', '/home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_16-44-01_042364_run_000000030', '/home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_16-44-01_042364_run_000000046', '/home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-03_03-23-26_500136_run_000000008', '/home/enrico/ds/performance_modelling/output/test_local_planni

In [22]:
p_test = ['/home/enrico/tmp/test/session_2022-01-18_12-10-42_573626_run_000000000',
'/home/enrico/tmp/test/session_2022-01-18_12-44-09_873705_run_000000000',
'/home/enrico/tmp/test/session_2022-01-18_12-44-09_873705_run_000000001',
'/home/enrico/tmp/test/session_2022-01-18_12-44-09_873705_run_000000002',
'/home/enrico/tmp/test/session_2022-01-18_12-44-09_873705_run_000000003']
print(p_test)

['/home/enrico/tmp/test/session_2022-01-18_12-10-42_573626_run_000000000', '/home/enrico/tmp/test/session_2022-01-18_12-44-09_873705_run_000000000', '/home/enrico/tmp/test/session_2022-01-18_12-44-09_873705_run_000000001', '/home/enrico/tmp/test/session_2022-01-18_12-44-09_873705_run_000000002', '/home/enrico/tmp/test/session_2022-01-18_12-44-09_873705_run_000000003']


In [28]:
for p in p_real:
    if path.exists(p):
        !mv {p} -t ~/tmp/test_local_planning_ex_rpp/
        print("mv", p, "~/tmp/test_local_planning_ex_rpp/")
    else:
        print("file not found", p)

mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_10-33-14_965419_run_000000070 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_13-54-20_413082_run_000000016 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_16-44-01_042364_run_000000015 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_16-44-01_042364_run_000000018 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_16-44-01_042364_run_000000029 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_16-44-01_042364_run_000000030 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-02_16-44-0

mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-03_03-23-26_500136_run_000000616 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-03_03-23-26_500136_run_000000620 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-03_03-23-26_500136_run_000000626 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-03_03-23-26_500136_run_000000639 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-03_03-23-26_500136_run_000000655 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-03_03-23-26_500136_run_000000674 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-03_03-23-2

mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-04_23-03-28_247570_run_000000184 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-04_23-03-28_247570_run_000000207 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-04_23-03-28_247570_run_000000219 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-04_23-03-28_247570_run_000000225 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-04_23-03-28_247570_run_000000229 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-04_23-03-28_247570_run_000000245 ~/tmp/test_local_planning_ex_rpp/
mv /home/enrico/ds/performance_modelling/output/test_local_planning/session_2021-11-04_23-03-2